In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import random
from collections import deque, Counter
import pygame

# Mapping dictionary
gestures = {
    0: 'Rock',
    1: 'Paper',
    2: 'Scissors'
}

# Scoring system
score = 0
round_number = 0

# For storing the last 10 gestures
last_gestures = deque(maxlen=10)

# Initialize mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Initialize Pygame
pygame.init()
win = pygame.display.set_mode((500, 400))

# Initialize camera
cap = cv2.VideoCapture(0)

def detect_gesture_from_landmarks(landmarks):
    thumb_is_open = landmarks[mp_hands.HandLandmark.THUMB_TIP.value][0] > landmarks[mp_hands.HandLandmark.THUMB_MCP.value][0]
    fingers = [mp_hands.HandLandmark.INDEX_FINGER_TIP.value, mp_hands.HandLandmark.MIDDLE_FINGER_TIP.value,
               mp_hands.HandLandmark.RING_FINGER_TIP.value, mp_hands.HandLandmark.PINKY_TIP.value]
    fingers_open = [landmarks[finger][1] < landmarks[mp_hands.HandLandmark.WRIST.value][1] for finger in fingers]
    
    if thumb_is_open and all(fingers_open):
        return 1  # Paper
    elif not thumb_is_open and all(fingers_open):
        return 2  # Scissors
    elif not thumb_is_open and not any(fingers_open):
        return 0  # Rock
    else:
        return None  # Unclear gesture


# Game loop
with mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Flip the image horizontally for a better selfie
        image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        
        # To improve performance, make the image not writable
        image.flags.writeable = False
        results = hands.process(image)
        
        # Draw hand landmarks
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        # If there is a hand detected
        if results.multi_hand_landmarks is not None:
            # Get the hand landmarks
            landmarks = list(np.array([[landmark.x, landmark.y, landmark.z] 
                        for landmark in results.multi_hand_landmarks[0].landmark]))

            # Detect hand gesture
            gesture_code = detect_gesture_from_landmarks(landmarks)

            if gesture_code is not None:  # Only process clear gestures
                last_gestures.append(gesture_code)

                # Only proceed if we have 10 gestures and they're all the same
                if len(last_gestures) == 10 and len(set(last_gestures)) == 1:
                    gesture_code = last_gestures[0]
                    gesture_name = gestures[gesture_code]

                    # Get computer's move and calculate score
                    computer_move = random.choice(list(gestures.values()))
                    if (gesture_name == 'Rock' and computer_move == 'Scissors') or \
                    (gesture_name == 'Paper' and computer_move == 'Rock') or \
                    (gesture_name == 'Scissors' and computer_move == 'Paper'):
                        score += 1
                        result_text = "You won!"
                        color = (0, 255, 0)
                    elif gesture_name == computer_move:
                        result_text = "It's a tie!"
                        color = (255, 255, 255)
                    else:
                        result_text = "You lost!"
                        color = (0, 0, 255)

                    round_number += 1

                    # Clear last gestures
                    last_gestures.clear()

                    # Show game info in Pygame window
                    win.fill((0, 0, 0))
                    font = pygame.font.Font(None, 36)
                    text = font.render("Round: {}".format(round_number), 1, (255, 255, 255))
                    win.blit(text, (20, 20))
                    text = font.render("Your move: {}".format(gesture_name), 1, (255, 255, 255))
                    win.blit(text, (20, 60))
                    text = font.render("Computer's move: {}".format(computer_move), 1, (255, 255, 255))
                    win.blit(text, (20, 100))
                    text = font.render("Score: {}".format(score), 1, (255, 255, 255))
                    win.blit(text, (20, 140))
                    text = font.render(result_text, 1, color)
                    win.blit(text, (20, 180))
                    pygame.display.flip()

        # Show the image from camera
        cv2.imshow('Game', image)

        # Break the loop on 'q' key press
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()
pygame.quit()
